In [ ]:
from kstar import config
config.install_resource_files()
#The command below will install the necessary network files for KSTAR to run. 
config.install_network_files()

In [ ]:
from kstar import config

#update network directory: If KSTAR does not find this directory + necessary files, it will notify you
config.NETWORK_DIR, config.NETWORK_Y_PICKLES, config.NETWORK_ST_PICKLES = config.update_network_directory('./NETWORKS/NetworKIN')

#create network pickles. Only need to run this function if network pickles were not already generated.
config.create_network_pickles()

In [ ]:
from kstar import config

config.check_configuration()

# Map Datasets to KinPred

## Download and Process Dataset of Interest
Prior to predicting kinase activities, datasets need to be mapped to KinPred to obtain the Uniprot ID, phosphosite, and the +/-7 peptide sequence that will be used by KSTAR to identify which kinases are associated with each phosphosite. In order to map kinase activities, the dataframe containing phosphoproteomic data should contain each peptides Uniprot accession, as well as either the site number or peptide sequence. If the peptide sequence is used, it should be formatted with only the phosphorylated peptides being lowercased. For example, if a peptide sequence is annotated with '(ph)' in front of the phosphorylated amino acid, you would need to remove the '(ph)' from the sequence and lowercase the phosphorylated amino acid. So, the peptide sequence SGLAYCPND(ph)YHQLFSPR would become SGLAYCPNDyHQLFSPR.

It is recommended to use the peptide sequence rather than the site number when possible, as this is more likely to be found in the most recent version of KinPred. An example of the processed dataset can be seen below, which is a trimmed, processed, and mapped version of the dataset published publically (Chylek, 2014). You can download this data from the original publication below or the pre-mapped version at [FigShare](https://figshare.com/articles/dataset/KSTAR_Supplementary_Data/14919726).

Reference:  L. A. Chylek, V. Akimov, J. Dengjel, K. T. G. Rigbolt, B. Hu, W. S. Hlavacek, and B. Blagoev.
Phosphorylation Site Dynamics of Early T-cell Receptor Signaling. PLoS ONE, 9(8):e104240,
2014.

In [1]:
#import KSTAR and other necesary packages
import pandas as pd
import numpy as np
import pickle
import os

from kstar import config, helpers, mapping

In [2]:
#load data
df = pd.read_csv('example.tsv', index_col = 0, sep = '\t')
df

Notice that all data columns in the dataset have 'data:' in front of them. This is how KSTAR will identify which columns to use when making evidence decisions. This can be done manually prior to mapping, or will be done by KSTAR automatically once you indicate which columns you would like to use as evidence.

## Map the Dataset to KinPred

First, we need to indicate where to save the mapped data and where to record information about the mapping process:

In [3]:
#define the directory where mapped dataset and run information will be saved. 
odir = './example'
#if directory doesn't exist yet, create it
if not os.path.exists(f"{odir}/MAPPED_DATA"): 
    os.mkdir(f"{odir}/MAPPED_DATA")   

Next, we want to intialize a log file which will record information about the mapping run, including any errors that arise during the process. This log file will be saved in the location indicated by the second parameter of the 'helpers.get_logger()' function.

In [4]:
#Define the log name
logName = 'example_run'
#intialize logger
mapping_log = helpers.get_logger(f"mapping_{logName}", f"{odir}/MAPPED_DATA/mapping_{logName}.log")

The last step before mapping your dataset to KinPred is to construct a dictionary which indicates the columns where KSTAR can find information about each peptide. This should include the 'accession_id', the uniprot accession corresponding to the identified peptide. It should also include __either__ the 'peptide' (amino acid sequence with phosphorylation sites lowercased) or the 'site' (modified amino acid + modification location, such as Y11). It is recommended to use the peptide sequence when possible.

The format of this dictionary should be: {'peptide': 'Col_with_PeptideInfo', 'accession_id': 'Col_with_UniprotID'}

In [5]:
mapDict = {'peptide':'peptide', 'accession_id':'query_accession'}

You are now ready to map the dataset, which can be done using the ExperimentMapper class.

In [6]:
#map dataset and record process in the logger
exp_mapper = mapping.ExperimentMapper(experiment = df,
                                      columns = mapDict, 
                                      logger = mapping_log)
#save mapped dataset
exp_mapper.experiment.to_csv(f"{odir}/MAPPED_DATA/{logName}_mapped.tsv", sep = '\t', index = False)

If you look at the ExperimentMapper class, you will find that five new columns have been added to the original dataset, which allows for easy mapping to KSTAR networks used in activity prediction.

In [7]:
exp_mapper.experiment.head()

These additional columns have the following meaning:

1. 'KSTAR_ACCESSION': Uniprot accession id corresponding to reviewed protein sequence, focusing only on the canonical isoforms of each protein.
2. 'KSTAR_PEPTIDE': Peptide sequence containing a single phosphorylation site, including the 7 amino acids both before and after the modified residue.
3. 'KSTAR_SITE': Location of modified residue in the protein
4. 'KSTAR_NUM_COMPENDIA': The number of different phosphoproteome compendia that modification site is identified in, used as an indicator of the study bias of each modification site. For this purpose, PhosphoSitePlus, PhosphoELM, HPRD, ______, and ProteomeScout were profiled.
5. 'KSTAR_NUM_COMPENDIA_CLASS': Same as 4, but sites are grouped into smaller classes based on study bias (0 is <1 compendia, 1 is 1-3 compendia, 2 is >3 compendia)

## Explore the Mapped Dataset

One way to check how well the mapping worked is to check the number of peptides in the original dataframe vs. the newly mapped dataframe. 

In [8]:
#Look at how many of the peptides in the dataset were actually mapped. This indicates how well the mapping worked
print("Original number of lines in df: %d\nNew number of lines in mapped:%d"%(len(df), len(exp_mapper.experiment)))

In this case, no peptides were lost during the mapping process. Changes to the number of peptides could be a result of several things:
1. Failure to find the corresponding peptide 
2. A peptide with multiple phosphorylation sites was seperated into seperate peptides

For specifics on mapping failures, go to the log file for details.

# Predict Kinase Activities

Once pruned networks have been generated or downloaded and data has been mapped, you are now ready to generate kinase activity predictions. Kinase activity calculation requires selection of the following choices

1. __phospho_type:__ Tyrosine ['Y'] or Serine/Threonine ['ST'], or both ['Y, 'ST'] (default). 
2. __agg:__ How to handle duplicates of the same peptide (i.e. aggregation) and determine sites to use as evidence. 
    1. 'count': counts the total number of times a non-NaN value of that peptide occurred in an experiment, and accepts any site that appears an amount of times equal to or above the threshold. This is recommended if you do not want to use quantification values and instead want to focus only on sites identified in each sample. 
    2. 'mean': averages the non-NaN values found for multiple peptides. We recommend using this when you do want to use quantification values, such as when comparing a pre and post treatment condition.
    3. 'max': will obtain the maximum quantification value seen for the given site
    4. 'min': will obtain the minimum quantification value seen for the given site
3. __threshold:__ defines which of the phosphorylation sites to use as evidence for each condition. If greater == True (default), any site with an aggregated value greater than the threshold will be used as evidence for prediction for each condition. If greater == False, any site with an aggregated value less than the threshold will be used as evidence for prediction for each condition.

In [9]:
#Import preamble of kstar and other necessary functions
import pandas as pd
import os
import pickle

from kstar import config, helpers, calculate_v1

## Determine Thresholds
It is useful to determine the best threshold to use (which sites to use as evidence for each sample). One easy benchmark way is to identify the number of sites that will be used as evidence at different thresholds. For tyrosine kinase activities, it is generally recommended that samples have at least 50 tyrosine sites used as evidence, while for serine/threonine kinase activities, it is recommended that samples have at least 1000 sites used as evidence. While not a necessity, it is also beneficial to have comparable site numbers across samples.

Similar to mapping, a log file will be generated to indicate the progress of activity prediction and any errors that arise during the process. To intialize the log file, run the below code. It can be useful to name your log files differently based on whether you are generating predictions for tyrosine kinases or serine/threonine kinases.

In [10]:
#define the directory where mapped dataset and run information will be saved. 
odir = './example'
logName = 'example_run'
#Name
logName_new = logName + '_Y'
#if directory does not exits, create it
if not os.path.exists(f"{odir}/RESULTS"): 
    os.mkdir(f"{odir}/RESULTS")
#intialize log file
activity_log = helpers.get_logger(f"activity_{logName_new}", f"{odir}/RESULTS/activity_{logName_new}.log")

In [11]:
#load mapped data, if necessary
experiment = pd.read_csv(f'{odir}/MAPPED_DATA/{logName}_mapped.tsv', sep = '\t', index_col = 0)

Next, we need to define the parameters to use for the generation of evidence in this experiment. These parameters include:

1. __data_columns__: the location of abundance values from phosphoproteomic experiment. Either can provide a list of column names, or if is None, KSTAR will look for any column that contains 'data:' at the beginning of the name. Since the example dataset includes this 'data:' format, we will set this parameter to None.

2. __agg__: We would like phosphorylation sites select sites based on the their abundance relative to the pre-stimulation condition, with sites appearing multiple times aggregated using a mean. As such, we will set this to be 'mean'. Other options include 'max' or 'min', which will take the maximum or minimum value seen across multiple sites. Last option is 'count', which will count the number of times a site appears across the dataset. This is good if you would like to use all sites observed in a sample (set threshold to 1).

3. __greater__: boolean that indicates whether sites greater than or less than the threshold should be kept. We are interested in knowing which kinases activate upon TCR stimulation, so we want to look at phosphorylation sites with phosphorylation. For this reason, we will set greater = True.

4. __threshold__: This will define the cutoff for keeping a phosphorylation site as evidence. If we look at the original data, abundance values are relative to the 0 minute timepoint and appear to be log transformed. We want to focus on kinases that have increased activity after stimulation, so we think that 0.2 (phosphorylation sites logfold change >0.2 used as evidnece) will be  a good threshold.

5. __evidence_size__: Optional, if provided will override threshold. Instead of using a threshold, this parameter will force all samples to use the same number of phosphorylation sites as evidence for prediction for each sample. For example, if evidence_size = 100 and greater = True, the 100 sites with the highest quantification values are pulled for each sample. This approach has not been extensively tested, so we only recommend this approach if thresholding produces a wide variety of evidence sizes.

In [12]:
#Define parameters
data_columns = None
agg = 'mean' 
greater = True
threshold = 0.2

For all activity prediction steps, the KinaseActivity class (found within the calculate module) is used. To initialize the class, you only need to provide the original experiment, the logger, the data columns with abundance information, and the modification type of interest. We can then use the test_threshold() function in the KinaseActivity class to determine the number sites that will be used as evidence in each condition given a threshold of 0.2.

In [13]:
#intialize KinaseActivity class object
kinact = calculate_v1.KinaseActivity(experiment, activity_log,data_columns = data_columns, phospho_type='Y')
#convert evidence into binary evidence based on the provided threshold
evidence_sizes = kinact.test_threshold(agg = agg, threshold = threshold,  greater = True, return_evidence_sizes = True)

The test_threshold function will output basic statistics on evidence sizes used. Here, we can see that with a threshold of 0.2, the minimum number of sites used is 128, which is above our recommended minimum of 50. We also see that the difference in evidence size is not large, so samples should be comparable. If you want to further inspect the data, the test_threshold() function has the option to return evidence sizes for each individual experiment:

In [14]:
#inspect the number of sites used for each sample
evidence_sizes

All samples above (except for the 0 time point, which will be removed prior to activity calculation) have greater than 50 sites being used as evidence, and the evidence sizes are generally comparable. This is a good threshold to use.

## Calculate Statistical Enrichment (Hypergeometric p-values)

Now that we have defined our parameters, the first step to obtaining activity predictions is calculating the statistical enrichment of kinase substrates in the dataset using the hypergeometric test. For each network + kinase, the hypergeometric tests asks the likelihood that k kinase substrates were identified in an experiment with n identified peptides, given that there are K kinase substrates in the entire phosphoproteome, N. From these tests, a set of 50 p-values for each kinase will be generated indicating the enrichment found in each pruned network. To obtain these predictions, use the following steps

1. Load the pruned kinase-substrate networks from the network pickles. Since the example dataset provided here is tyrosine enriched, using tyrosine networks. 

In [15]:
phospho_types = ['Y'] #running on this type of kinase/substrate network

#Load the pickles containing the 50 pruned networks for tyrosine kinases
networks = {}
networks['Y'] = pickle.load(open(config.NETWORK_Y_PICKLE, "rb" ) )

2. Initialize an activity log, which will store information about each run, including any errors that arise

In [16]:
#Create activity log: if already did this, ignore.
if not os.path.exists(f"{odir}/RESULTS"): 
    os.mkdir(f"{odir}/RESULTS")
activity_log = helpers.get_logger(f"activity_{logName}", f"{odir}/RESULTS/activity_{logName}.log")

3. Define the parameter values to use for activity calculation. These are the same as those described in the 'Determine Threshold' section for generating binary evidence.

In [17]:
data_columns = None #by passing None, all columns prefixed by data: will be used to calculate activity
agg = 'mean'
threshold = 0.2
greater = True

4. Perform enrichment calculations. The enrichment_analysis() function within the calculate module can be used to generate the statistical enrichment values. This will return a dictionary containing a different KinaseActivity object for each modification type with predictions ('Y' and/or 'ST).

In [18]:
kinact_dict = calculate_v1.enrichment_analysis(experiment, activity_log, networks, phospho_types = phospho_types, 
                                                agg =agg, threshold = threshold,  
                                                greater = greater, PROCESSES = 1)

You can view the median p-values obtained for each kinase by looking at 'activities' attribute of the KinaseActivity class. We recommend continuing to the subsequent sections to generate more robust activity predictions, as the we have found that the median p-values suffer from kinase-specific false positive rates.

## Generate random datasets, run kinase activity on random datasets, normalize original analysis

Once the enrichment of each kinase in the pruned networks has been calculated from the enrichment_analysis() function, we would next like to ask, for each kinase, how likely it is to have obtained that same enrichment (as indicated by the p-value) or better from a random phosphoproteomic experiment. 

To achieve this, we have two approaches:

1. Generating Random Phosphoproteomic Experiments: This method involves simulating random phosphoproteomic datasets and performing hypergeometric enrichment on these experiments. We have found that generating 150 random experiments provides a good balance between statistical power and computational complexity.
2. Using Pregenerated Random Experiments: To save computational resources and time, precomputed random experiment activity lists can be used instead. This approach eliminates the need for on-the-fly random dataset generation, significantly improving efficiency, particularly for large datasets.


*Note: For larger datasets this can be slow and memory intensive. In order to speed up these calculations, it is possible to use multiprocessing by adjusting the 'PROCESSES' parameter to the number of core you would like to use. The default is 1 (use only a single core). You can also use a highly parallelized version through nextflow (see 'KSTAR in Parallel')*

Using Pregenerated Random Experiment Activity Lists (Default)

KSTAR is set by default to use pregenerated random experiment activity lists to improve computational efficiency. If you wish to use pregenerated random experiments, configure the following parameters:

1. use_pregen_data = True: Enables the use of pregenerated random experiment activity lists, avoiding the need to generate new random experiments.
2. pregenerated_experiments_path (default: None): Specifies the file path to pregenerated random experiment activity lists. If left as None, KSTAR will use the default pregenerated activity lists.
3. network_hash (default: None): A unique identifier for the network used to generate the pregenerated random experiments. If using your own pregenerated experiment activity lists, you must set this to the unique ID (hash) of the network generated during pruning.
4. save_random_experiments = True (optional): If enabled, the full random experiment files used in the analysis will be saved for further examination.
    

Generating New Random Experiments
If you wish to generate your own random experiments instead of using pregenerated data, update the following parameters:

1. use_pregen_data = False: Disables the use of pregenerated random experiment activity lists, triggering the generation of new random experiments.
2. save_new_precompute = True (optional): Enables saving newly generated random experiment activity lists for future use. This allows the reuse of computed data in subsequent runs without needing to regenerate experiments. This is not required if you prefer to use KSTAR’s default pregenerated activity lists.
3. save_random_experiments = True (optional): If enabled, the full random experiment files generated during the process will be saved for further analysis.
By default, num_random_experiments = 150, providing a balance between statistical power and computational efficiency.

In [19]:
use_pregen_data = True 
save_new_precompute = False
pregenerated_experiments_path = None
directory_for_save_precompute = None
network_hash = None
save_random_experiments = False

In [20]:

#Set the number of random experiments
num_random_experiments=150

#Generate random experiments or use pregenerated random experiments
calculate_v1.randomized_analysis(
    kinact_dict,
    activity_log,
    num_random_experiments,
    use_pregen_data,
    save_new_precompute,
    pregenerated_experiments_path,
    directory_for_save_precompute,
    network_hash,
    save_random_experiments,
    PROCESSES=1
)

The random experiments generated from the above function can be found in the 'random_experiments' attribute of the KinaseActivity class. For each random experiment, the sites that will be included for random activity generation are indicated with 1.

In [21]:
#This object holds the random datasets that were used, where NaN means a site was not selected and 1 means it was
kinact_dict['Y'].random_experiments.head()

If you sum the random experiments object, it will tell you how many total sites were selected for each random experiment, which should match the parent experiment:

In [22]:
# If you sum the random_experiments it will tell you how many total sites were selected, which should match
# the parent experiment 
kinact_dict['Y'].random_experiments.sum()

## Calculate Mann Whitney Significance

Finally, final activity scores are calculated by implementing the Mann Whitney U-test to compare the distribution of real p-values (activity predictions on the real dataset) to the random p-values (activity predictions on the random dataset). Further, a false positive rate is calculated by pulling out one of the random datasets and comparing to all other random datasets using the same Mann Whitney test. The parameter 'number_sig_trials' indicates the number of times to repeat this calculation to obtain the false positive rate. As with the previous analysis, the 'PROCESSES' parameter can be adjusted to perform multiprocessing.

In [23]:
calculate_v1.Mann_Whitney_analysis(kinact_dict, activity_log, number_sig_trials = 100, PROCESSES = 1)

## Save KSTAR Results

There are a couple of options to save the results of KSTAR analysis. To save all information (hypergeometric enrichment results, random experiments, mann whitney activities and fpr), we have provided a function to save all information while not taking up too much memory.

In [24]:
calculate_v1.save_kstar_slim(kinact_dict, logName, odir)